In [4]:
import pandas as pd
import pulp

In [5]:
demand_df = pd.read_csv("data/demand.csv")
vehicles_df = pd.read_csv("top_ranked_combined.csv")
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
current_fleet = pd.read_csv("current_fleet.csv")

In [6]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         256 non-null    int64 
 1   Size         256 non-null    object
 2   Distance     256 non-null    object
 3   Demand (km)  256 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 8.1+ KB


In [7]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Allocation               256 non-null    object 
 1   Operating Year           256 non-null    int64  
 2   Size                     256 non-null    object 
 3   Distance_demand          256 non-null    object 
 4   Demand (km)              256 non-null    int64  
 5   ID                       256 non-null    object 
 6   Vehicle                  256 non-null    object 
 7   Available Year           256 non-null    int64  
 8   Cost ($)                 256 non-null    int64  
 9   Yearly range (km)        256 non-null    int64  
 10  Distance_vehicle         256 non-null    object 
 11  Fuel                     256 non-null    object 
 12  carbon_emissions_per_km  256 non-null    float64
 13  insurance_cost           256 non-null    float64
 14  maintenance_cost         2

In [8]:
carbon_limits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Year                    16 non-null     int64
 1   Carbon emission CO2/kg  16 non-null     int64
dtypes: int64(2)
memory usage: 388.0 bytes


In [9]:
import pulp
import pandas as pd
from typing import Dict, List

class FleetPlanningLP:
    def __init__(self, 
                 demand_df: pd.DataFrame,
                 vehicles_df: pd.DataFrame,
                 carbon_limits_df: pd.DataFrame):
        """
        Initialize the Fleet Planning LP model with DataFrame inputs.
        
        Args:
            demand_df: DataFrame with columns [Year, Size, Distance, Demand (km)]
            vehicles_df: DataFrame with vehicle specifications
            carbon_limits_df: DataFrame with yearly carbon emission limits
        """
        # Store input data
        self.demand_df = demand_df
        self.vehicles_df = vehicles_df
        self.carbon_limits_df = carbon_limits_df
        
        # Extract unique values
        self.years = sorted(demand_df['Year'].unique())
        self.size_buckets = sorted(demand_df['Size'].unique())
        self.distance_buckets = sorted(demand_df['Distance'].unique())
        self.vehicle_types = sorted(vehicles_df['Vehicle'].unique())
        
        # Create vehicle type lookup dictionaries
        self.vehicle_costs = self._create_vehicle_costs_dict()
        self.vehicle_ranges = self._create_vehicle_ranges_dict()
        self.vehicle_emissions = self._create_vehicle_emissions_dict()
        
        # Initialize the LP problem
        self.problem = pulp.LpProblem("Fleet_Planning", pulp.LpMinimize)
        
        # Initialize decision variables
        self._init_decision_variables()
        
    def _create_vehicle_costs_dict(self) -> Dict:
        """Create dictionary of vehicle costs."""
        costs = {}
        for _, row in self.vehicles_df.drop_duplicates('Vehicle').iterrows():
            vehicle = row['Vehicle']
            costs[vehicle] = {
                'purchase': row['Cost ($)'],
                'operating': (row['insurance_cost'] + 
                            row['maintenance_cost'] + 
                            row['fuel_costs_per_km'] * row['Yearly range (km)']),
                'selling': row['Cost ($)'] * 0.5  # Assuming 50% residual value
            }
        return costs
    
    def _create_vehicle_ranges_dict(self) -> Dict:
        """Create dictionary of vehicle yearly ranges."""
        return dict(zip(
            self.vehicles_df['Vehicle'].unique(),
            self.vehicles_df.drop_duplicates('Vehicle')['Yearly range (km)']
        ))
    
    def _create_vehicle_emissions_dict(self) -> Dict:
        """Create dictionary of vehicle emissions per km."""
        return dict(zip(
            self.vehicles_df['Vehicle'].unique(),
            self.vehicles_df.drop_duplicates('Vehicle')['carbon_emissions_per_km']
        ))
            
    def _init_decision_variables(self):
        """Initialize the decision variables for the LP model."""
        # Number of vehicles of type v used in year t
        self.x = pulp.LpVariable.dicts("fleet",
            ((v, t) for v in self.vehicle_types for t in self.years),
            lowBound=0,
            cat='Integer')
        
        # Number of new vehicles bought
        self.b = pulp.LpVariable.dicts("buy",
            ((v, t) for v in self.vehicle_types for t in self.years),
            lowBound=0,
            cat='Integer')
        
        # Number of vehicles sold
        self.s = pulp.LpVariable.dicts("sell",
            ((v, t) for v in self.vehicle_types for t in self.years),
            lowBound=0,
            cat='Integer')
    
    def build_model(self):
        """Build the complete LP model with all constraints."""
        self._add_objective_function()
        self._add_demand_constraints()
        self._add_fleet_balance_constraints()
        self._add_emission_constraints()
        self._add_vehicle_life_constraints()
        self._add_selling_limit_constraints()
    
    def _add_objective_function(self):
        """Add the objective function to minimize total costs."""
        objective = pulp.lpSum(
            self.vehicle_costs[v]['purchase'] * self.b[v, t] +
            self.vehicle_costs[v]['operating'] * self.x[v, t] -
            self.vehicle_costs[v]['selling'] * self.s[v, t]
            for v in self.vehicle_types
            for t in self.years
        )
        self.problem += objective
    
    def _add_demand_constraints(self):
        """Add demand satisfaction constraints."""
        for _, row in self.demand_df.iterrows():
            year = row['Year']
            size = row['Size']
            distance = row['Distance']
            demand = row['Demand (km)']
            
            # Filter suitable vehicles for this size and distance combination
            suitable_vehicles = self.vehicles_df[
                (self.vehicles_df['Size'] == size) &
                (self.vehicles_df['Distance_vehicle'] == distance)
            ]['Vehicle'].unique()
            
            if len(suitable_vehicles) > 0:
                self.problem += (
                    pulp.lpSum(self.x[v, year] * self.vehicle_ranges[v] 
                             for v in suitable_vehicles) >= demand,
                    f"Demand_{size}_{distance}_{year}"
                )
    
    def _add_fleet_balance_constraints(self):
        """Add fleet balance constraints."""
        start_year = min(self.years)
        
        # First year constraint
        for v in self.vehicle_types:
            self.problem += (
                self.x[v, start_year] == self.b[v, start_year],
                f"Initial_Fleet_Balance_{v}"
            )
        
        # Subsequent years
        for v in self.vehicle_types:
            for t in range(start_year + 1, max(self.years) + 1):
                self.problem += (
                    self.x[v, t] == self.x[v, t-1] + self.b[v, t] - self.s[v, t],
                    f"Fleet_Balance_{v}_{t}"
                )
    
    def _add_emission_constraints(self):
        """Add carbon emission limit constraints."""
        for _, row in self.carbon_limits_df.iterrows():
            year = row['Year']
            emission_limit = row['Carbon emission CO2/kg']
            
            total_emissions = pulp.lpSum(
                self.x[v, year] * self.vehicle_emissions[v] * self.vehicle_ranges[v]
                for v in self.vehicle_types
            )
            
            self.problem += (
                total_emissions <= emission_limit,
                f"Emission_Limit_{year}"
            )
    
    def _add_vehicle_life_constraints(self):
        """Add 10-year vehicle life limit constraints."""
        for v in self.vehicle_types:
            for t in range(2035, max(self.years) + 1):
                self.problem += (
                    self.s[v, t] >= self.b[v, t-10],
                    f"Vehicle_Life_{v}_{t}"
                )
    
    def _add_selling_limit_constraints(self):
        """Add maximum 20% selling limit constraints."""
        start_year = min(self.years)
        for v in self.vehicle_types:
            for t in range(start_year + 1, max(self.years) + 1):
                self.problem += (
                    self.s[v, t] <= 0.2 * self.x[v, t-1],
                    f"Selling_Limit_{v}_{t}"
                )
    
    def solve(self) -> bool:
        """
        Solve the LP model.
        
        Returns:
            bool: True if optimal solution found, False otherwise
        """
        status = self.problem.solve()
        return status == pulp.LpStatusOptimal
    
    def get_results(self) -> pd.DataFrame:
        """
        Get the results as a pandas DataFrame.
        
        Returns:
            DataFrame containing the optimal fleet plan
        """
        results = []
        for v in self.vehicle_types:
            vehicle_data = self.vehicles_df[self.vehicles_df['Vehicle'] == v].iloc[0]
            for t in self.years:
                results.append({
                    'Year': t,
                    'Vehicle': v,
                    'Size': vehicle_data['Size'],
                    'Distance': vehicle_data['Distance_vehicle'],
                    'Fleet_Size': self.x[v, t].value(),
                    'Vehicles_Bought': self.b[v, t].value(),
                    'Vehicles_Sold': self.s[v, t].value(),
                    'Purchase_Cost': self.vehicle_costs[v]['purchase'],
                    'Operating_Cost': self.vehicle_costs[v]['operating'],
                    'Yearly_Range': self.vehicle_ranges[v],
                    'Emissions_per_km': self.vehicle_emissions[v]
                })
        return pd.DataFrame(results)

In [11]:
# Create and solve the model
model = FleetPlanningLP(demand_df, vehicles_df, carbon_limits_df)
model.build_model()

model.solve()
# Get results
results_df = model.get_results()

# Calculate some summary statistics
yearly_summary = results_df.groupby('Year').agg({
    'Fleet_Size': 'sum',
    'Vehicles_Bought': 'sum',
    'Vehicles_Sold': 'sum'
}).reset_index()

print("Yearly Fleet Summary:")
print(yearly_summary)

# Calculate total costs
total_purchase_cost = (results_df['Vehicles_Bought'] * results_df['Purchase_Cost']).sum()
total_operating_cost = (results_df['Fleet_Size'] * results_df['Operating_Cost']).sum()

print(f"\nTotal Purchase Cost: ${total_purchase_cost:,.2f}")
print(f"Total Operating Cost: ${total_operating_cost:,.2f}")

Yearly Fleet Summary:
    Year  Fleet_Size  Vehicles_Bought  Vehicles_Sold
0   2023         0.0              0.0            0.0
1   2024         0.0              0.0            0.0
2   2025         0.0              0.0            0.0
3   2026         0.0              0.0            0.0
4   2027         0.0              0.0            0.0
5   2028         0.0              0.0            0.0
6   2029         0.0              0.0            0.0
7   2030         0.0              0.0            0.0
8   2031         0.0              0.0            0.0
9   2032         0.0              0.0            0.0
10  2033         0.0              0.0            0.0
11  2034         0.0              0.0            0.0
12  2035         0.0              0.0            0.0
13  2036         0.0              0.0            0.0
14  2037         0.0              0.0            0.0
15  2038         0.0              0.0            0.0

Total Purchase Cost: $0.00
Total Operating Cost: $0.00


In [22]:
model = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)

In [23]:
# Decision variables
buy_vars = {}   # Vehicles bought each year
use_vars = {}   # Vehicles in use each year
sell_vars = {}  # Vehicles sold each year

In [24]:
vehicles_topsis_combinations = vehicles_df.groupby(['Operating Year', 'Size', 'Distance_demand'])

In [25]:
current_fleet_combinations = current_fleet.groupby(['Size_Bucket', 'Distance_Bucket'])

In [45]:
for (year, size, distance), group in vehicles_topsis_combinations:
    vehicle_id_list = group["ID"].tolist()  # Extract vehicle IDs from the group

    # Initialize the year if not already in dictionary
    if year.item() not in buy_vars:
        buy_vars[year.item()] = {}

    # Initialize the (size, distance) key within the year
    if f"{size}_{distance}" not in buy_vars[year.item()]:
        buy_vars[year.item()][f"{size}_{distance}"] = {}

    # Create decision variables for each vehicle ID in this (size, distance) combination
    for v_id in vehicle_id_list:
        buy_vars[year][f"{size}_{distance}"] = pulp.LpVariable(f"{v_id}", lowBound=0, cat="Integer")


In [46]:
for index, row in current_fleet.iterrows():
    size = row["Size_Bucket"]
    distance = row["Distance_Bucket"]
    v_id = row["IDs"]
    sell_vars[f"{size}_{distance}"] = pulp.LpVariable(f"{v_id}", lowBound=0, cat="Integer")

In [47]:
buy_vars

{2023: {'S1_D1': LNG_S1_2023,
  'S1_D2': Diesel_S1_2023,
  'S1_D3': Diesel_S1_2023,
  'S1_D4': Diesel_S1_2023,
  'S2_D1': LNG_S2_2023,
  'S2_D2': Diesel_S2_2023,
  'S2_D3': Diesel_S2_2023,
  'S2_D4': Diesel_S2_2023,
  'S3_D1': LNG_S3_2023,
  'S3_D2': Diesel_S3_2023,
  'S3_D3': Diesel_S3_2023,
  'S3_D4': Diesel_S3_2023,
  'S4_D1': LNG_S4_2023,
  'S4_D2': Diesel_S4_2023,
  'S4_D3': Diesel_S4_2023,
  'S4_D4': Diesel_S4_2023},
 2024: {'S1_D1': LNG_S1_2024,
  'S1_D2': Diesel_S1_2024,
  'S1_D3': Diesel_S1_2024,
  'S1_D4': Diesel_S1_2024,
  'S2_D1': LNG_S2_2024,
  'S2_D2': Diesel_S2_2024,
  'S2_D3': Diesel_S2_2024,
  'S2_D4': Diesel_S2_2024,
  'S3_D1': LNG_S3_2024,
  'S3_D2': Diesel_S3_2024,
  'S3_D3': Diesel_S3_2024,
  'S3_D4': Diesel_S3_2024,
  'S4_D1': LNG_S4_2024,
  'S4_D2': Diesel_S4_2024,
  'S4_D3': Diesel_S4_2024,
  'S4_D4': Diesel_S4_2024},
 2025: {'S1_D1': LNG_S1_2025,
  'S1_D2': Diesel_S1_2025,
  'S1_D3': Diesel_S1_2025,
  'S1_D4': Diesel_S1_2025,
  'S2_D1': LNG_S2_2025,
  'S2_D2': 

In [37]:
def create_objective_function(model, buy_vars, use_vars, sell_vars, vehicles_df, years_range):
    """
    Creates the objective function for fleet optimization problem
    
    Parameters:
    - model: pulp.LpProblem object
    - buy_vars: Dictionary of purchase decision variables
    - use_vars: Dictionary of usage decision variables
    - sell_vars: Dictionary of sell decision variables
    - vehicles_df: DataFrame containing vehicle information
    - years_range: Range of years to consider
    """
    
    # Initialize total cost
    total_cost = 0
    
    # Purchase costs
    for year in years_range:
        for size_distance, vehicles in buy_vars[year].items():
            for v_id, var in vehicles.items():
                # Get purchase cost for this vehicle from vehicles_df
                purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Cost ($)'].values[0]
                total_cost += purchase_cost * var

    # # Operating costs for vehicles in use
    # for year in years_range:
    #     for size_distance, vehicles in use_vars[year].items():
    #         for v_id, var in vehicles.items():
    #             # Get operating cost for this vehicle from vehicles_df
    #             operating_cost = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Operating_Cost'].values[0]
    #             yearly_distance = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Yearly_Distance'].values[0]
    #             total_cost += operating_cost * yearly_distance * var

    # # Salvage value (negative cost) for sold vehicles
    # for size_distance, vehicles in sell_vars.items():
    #     for v_id, var in vehicles.items():
    #         # Get salvage value for this vehicle from vehicles_df
    #         # Assuming salvage value is a percentage of purchase cost
    #         purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Purchase_Cost'].values[0]
    #         salvage_value = 0.2 * purchase_cost  # Example: 20% of purchase cost
    #         total_cost -= salvage_value * var

    # Set the objective function
    model += total_cost

    return model

In [38]:
model = create_objective_function(model, buy_vars, use_vars, sell_vars, vehicles_df, range(2025, 2039))